In [ ]:
import random
import json
import pickle
import numpy as np
import nltk
from nltk.stem import WordNetLemmatizer
import tensorflow

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation, Dropout
from tensorflow.keras.optimizers import SGD

In [ ]:
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('omw-1.4')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...


True

In [ ]:
lemmatizer = WordNetLemmatizer() # create an instance of WordNetLemmatizer

intents = json.loads(open('/content/intents.json').read())

words = []
classes = []
documents = []
ignore_letters = ['?','!','.',',']

for intent in intents['intents']:
  for pattern in intent['patterns']:
    word_list = nltk.word_tokenize(pattern)  # tokenize the sentence i.e. "I am John" to "I","am","John"
    words.extend(word_list) #appending the tokenized word in the word_list
    documents.append((word_list,intent['tag'])) # in documents we stored the the tokenized form of patterns along with the tag.
    if(intent['tag'] not in classes): 
      classes.append(intent['tag'])  #storing the unique tags in the classes list
# print(documents)
#lemmatize 
words = [lemmatizer.lemmatize(word) for word in words if word not in ignore_letters] # lemmatize the word to its simplest meaningful form
words = sorted(set(words)) # removing the duplicates in words
# print(words)
 
classes = sorted(set(classes)) # removing the duplicates classes (mostly not req.coz classes are unique and user defined)

pickle.dump(words, open('words.pkl','wb'))
pickle.dump(classes, open('classes.pkl','wb'))


In [ ]:
# converting the textual data into numeric data to feed into the neural network
training =[]
output_empty = [0] * len(classes) # initialized the list with the number of tags in intents.json

for document in documents:  # document = (['How','are','you'],'greeting')
  bag = []
  word_patterns = document[0]  # document[0] consists of tokens eg. ['How','are','you']
  word_patterns = [lemmatizer.lemmatize(word.lower()) for word in word_patterns] # lemmatizing the word along with converting it in lowercase.
  for word in words: # words consists of tokens in lower as well as upper case.
    bag.append(1) if word in word_patterns else bag.append(0) # bag is created with columns equal to size of words list 
    # 1 is appended where there is no need to check for mapping since the words are already in lowercase. 
  output_row = list(output_empty)
  output_row[classes.index(document[1])] = 1  # document[1] consists of tags of that particular set of tokens
  # output_row store 1 mapping to index of tag in classes.
  training.append([bag,output_row]) #input is bag(tokens) and output is output_row(tags)


random.shuffle(training)
training = np.array(training)

train_x = list(training[:,0]) # data corresponding to all rows and just the first column(bag)
train_y = list(training[:,1]) # data corresponding to all rows and just the second column(output_row)

# training of model8
model = Sequential()
model.add(Dense(128, input_shape=(len(train_x[0]),),activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(len(train_y[0]),activation= 'softmax'))

sgd = SGD(lr= 0.01,decay= 1e-6,momentum=0.9,nesterov=True)
model.compile(loss='categorical_crossentropy',optimizer=sgd,metrics=['accuracy'])

hist = model.fit(np.array(train_x),np.array(train_y),epochs=200,batch_size=5,verbose=1)


<ipython-input-5-7b2c5f0eb7cb>:19: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  training = np.array(training)
/usr/local/lib/python3.8/dist-packages/keras/optimizers/optimizer_v2/gradient_descent.py:108: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(SGD, self).__init__(name, **kwargs)


Epoch 1/200
18/18 [==============================] - 1s 2ms/step - loss: 3.2635 - accuracy: 0.0341
Epoch 2/200
18/18 [==============================] - 0s 3ms/step - loss: 3.1388 - accuracy: 0.0455
Epoch 3/200
18/18 [==============================] - 0s 2ms/step - loss: 3.1022 - accuracy: 0.1364
Epoch 4/200
18/18 [==============================] - 0s 2ms/step - loss: 2.9743 - accuracy: 0.1477
Epoch 5/200
18/18 [==============================] - 0s 2ms/step - loss: 2.9320 - accuracy: 0.1705
Epoch 6/200
18/18 [==============================] - 0s 2ms/step - loss: 2.8050 - accuracy: 0.1932
Epoch 7/200
18/18 [==============================] - 0s 2ms/step - loss: 2.7076 - accuracy: 0.2273
Epoch 8/200
18/18 [==============================] - 0s 2ms/step - loss: 2.6139 - accuracy: 0.2045
Epoch 9/200
18/18 [==============================] - 0s 2ms/step - loss: 2.4529 - accuracy: 0.2614
Epoch 10/200
18/18 [==============================] - 0s 3ms/step - loss: 2.3972 - accuracy: 0.2727
Epoch 11/

In [ ]:
model.save('chatbot_model.h5',hist)
print('done')

done


CHATBOT 

In [ ]:
import random
import json
import pickle
import numpy as np
import nltk
from nltk.stem import WordNetLemmatizer
import tensorflow


In [ ]:
from tensorflow.keras.models import Sequential,load_model
from tensorflow.keras.layers import Dense, Activation, Dropout
from tensorflow.keras.optimizers import SGD

In [ ]:
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('omw-1.4')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


True

In [ ]:
lemmatizer = WordNetLemmatizer()

intents = json.loads(open('/content/intents.json').read())

words = pickle.load(open('/content/words.pkl','rb'))
classes = pickle.load(open('/content/classes.pkl','rb'))
model = load_model('/content/chatbot_model.h5')

In [ ]:
def clean_up_sentence(sentence):
  sentence_words = nltk.word_tokenize(sentence) # message by user converted to tokens.
  sentence_words = [lemmatizer.lemmatize(word) for word in sentence_words] # lemmatize the words to its grammatically simplest form.
  return sentence_words

In [ ]:
def bag_of_words(sentence):
  sentence_words = clean_up_sentence(sentence) # consisits of tokens created from users message
  bag = [0] * len(words) # initialized the list with tokens 
  for w in sentence_words: 
    for i,word in enumerate(words): # enumerate adds a counter to an iterable eg. (words,start=0)
      if(word == w):
        bag[i] = 1 # updating the bag to 1 where we get the user token and words(tokens) as equal 
  return np.array(bag)

In [ ]:
def predict_class(sentence):
  bow = bag_of_words(sentence)
  res = model.predict(np.array([bow]))[0]
  ERROR_THRESHOLD = 0.25 # to bypass a specified number of pointer errors without terminating
  results  = [[i,r] for i,r in enumerate(res) if r>ERROR_THRESHOLD]

  results.sort(key=lambda x: x[1],reverse=True)
  return_list = [] 
  for r in results:
    return_list.append({'intent':classes[r[0]],'probability':str(r[1])})
  return return_list

In [ ]:
def get_response(intents_list, intents_json): # comparing the two the output of predict_class and intents_json
  tag = intents_list[0]['intent']
  list_of_intents = intents_json['intents']
  for i in list_of_intents:
    if i['tag'] == tag: 
      result = random.choice(i['responses']) # if get matched output randomly from the responses
      break
  return result

In [ ]:
bye = ("cya", "See you later", "Goodbye","goodbye","good bye", "I am Leaving", "Have a Good day", "bye", "cao", "see ya")


In [ ]:

k=1
# global s
# s=0
while (k):
  message = input("")
  ints = predict_class(message)
  res = get_response(ints,intents)
  for i in bye:
    if message==i:
      k=0
  print(res)

Hi
1/1 [==============================] - 0s 87ms/step
Hi. How are you feeling today?
I am suffering from cold
1/1 [==============================] - 0s 21ms/step
You should take a sudafed tablet after your meal.
I took sudafed tablet 
1/1 [==============================] - 0s 20ms/step
Good, now take some rest and then let me know if you are feeling better after having sudafed tablet.
I am still not feeling well
1/1 [==============================] - 0s 18ms/step
Ok, its not just a normal cough, you should consult a doctor.
I want to consult a doctor
1/1 [==============================] - 0s 21ms/step
will you please book an appointment.
Can you book it for me
1/1 [==============================] - 0s 18ms/step
yes please state time and doctor's name in the following format; 'Patient Name:  Date:  Doctor: Hospital:
Patients Name: Avinash Date:30082002 Doctor: Abhi Hospital: Ruby
1/1 [==============================] - 0s 24ms/step
Hi. How are you feeling today?
I am suffering from coug